# Recitation 4

## SQL Pt 2

### Review: intro to SQL

SQL (**S**tructured **Q**uerying **L**anguage) is a domain-specific language used to manage and query data. Data is stored in tables. For example, here is a table called **customer** that stores my company's customer records.

| id | name | address | city | postal_code | country |
| --- | --- | --- | --- | --- | --- |
| 1 | Maria Anders | Obere Str. 57 | Berlin | 12209 | Germany |
| 2 | Ana Trujillo | Avda. de la Constitución 2222 | Mexico City | 05021 | Mexico |
| 3 | Antonio Moreno | Mataderos 2312 |	Mexico City | 05023	| Mexico |
| 4 | Thomas Hardy | 120 Hanover Sq. | London | WA1 1DP | UK |
| 5 | Christina Berglund | Berguvsvägen 8 | Luleå | S-958 22 | Sweden |

Each table is defined by its name (**customer**) and its columns (**id**, **name**, **address**, **city**, etc). Each table is populated by rows, each of which represents a data record and maps each of the table's columns to values.

Given a database consisting of one or more tables, SQL makes it easy for me to ask questions about my data. 

Suppose I'm curious about which cities in which my customers reside. I can make the query
```
SELECT id, name, city FROM customer
```
and my SQL engine will return the following table.

| id | name | city |
| --- | --- | --- |
| 1 | Maria Anders | Berlin | 
| 2 | Ana Trujillo | Mexico City |
| 3 | Antonio Moreno | Mexico City |
| 4 | Thomas Hardy | London |
| 5 | Christina Berglund | Luleå |

Now let's suppose my company is based in Mexico and I'm curious about the cities in which my *domestic* customers reside. I can make my query more specific with a `WHERE` clause
```
SELECT id, name, city FROM customer WHERE country = "Mexico"
```
to which my SQL engine returns

| id | name | city |
| --- | --- | --- |
| 2 | Ana Trujillo | Mexico City |
| 3 | Antonio Moreno | Mexico City |

> Aside: The true power of SQL becomes apparent when you start considering multiple tables. Suppose my company also has a database table storing all past orders.
>
> | id | customer_id | date |
> | --- | --- | --- |
> | 10308 |  2 | 1996-09-18 |
> | 10309 |  3 | 1996-09-19 |
> | 10310 |  4 | 1996-09-20 |
>
> I can use SQL to ask which of my orders came from Mexico and who made the orders. The query
> ```
> SELECT order.id, order.date, customer.name, customer.city
> FROM order JOIN customer ON order.customer_id = customer.id
> WHERE customer.country = "Mexico"
> ```
> returns the table
> 
> | order.id | order.date |  customer.name |  customer.city |
> | --- | --- | --- | --- |
> | 10308 | 1996-09-18 | Ana Trujillo | Mexico City |
> | 10309 | 1996-09-19 | Antonio Moreno | Mexico City |
>
> For now, we'll focus on SQL in the context of single tables.

### Review: extending SQL query grammar

Last week, we extended the grammar of the SQL query language. We started with a grammar that allowed you to express simple queries of the form `SELECT id, name, city FROM customer`.

In [4]:
sealed trait TableExpr
case class TableIdentifier(val s : String) extends TableExpr
case class SelectQuery(val columns : List[String], table : TableExpr) extends TableExpr

defined trait TableExpr
defined class TableIdentifier
defined class SelectQuery

Using Scala syntax, the query `SELECT id, name, city FROM customer` would be
```
SelectQuery(List("id", "name", "city"), TableIdentifier("customer"))
```

We extended this grammar to support an optional `WHERE` clause, allowing users to express queries of the form `SELECT id, name, city FROM t WHERE country = "Mexico"`

In [5]:
sealed trait TableExpr
case class TableIdentifier(val s : String) extends TableExpr
case class SelectQuery(val columns : List[String], table : TableExpr, /**/ where : WhereClause /**/) extends TableExpr

/**/
sealed trait WhereClause
case object None extends WhereClause
case class Some(c : CondExpr) extends WhereClause

sealed trait CondExpr
case object ConstTrue extends CondExpr
case object ConstFalse extends CondExpr
case class Eq(columnName : String, value : String) extends CondExpr
/**/

defined trait TableExpr
defined class TableIdentifier
defined class SelectQuery
defined trait WhereClause
defined object None
defined class Some
defined trait CondExpr
defined object ConstTrue
defined object ConstFalse
defined class Eq

### SQL engine: getting started

This week, we'll take a look at the underlying SQL engine that runs user queries.

Our SQL engine takes a query and returns the table represented by that query. Before implementing our engine, we need a runtime representation for a table, which is given below. You do *not* need to understand this code. All that you should note is that there is a `TableRow` class and a `Table` class that the SQL engine will use to hold data. We give examples how to interact with `TableRow` and `Table` further below.

In [6]:
// SKIP

import scala.collection.mutable.ListBuffer

class TableRow(data : Map[String, String]) {
    def selectColumns(columns : List[String]) : TableRow = {
        val selectedData = this.data.filterKeys(columns.contains(_))
        return TableRow.create(selectedData)
    }
    def get(columnName : String) : String = {
        return this.data(columnName)
    }
}
object TableRow {
    def create(data : Map[String, String]) : TableRow = {
        return new TableRow(data)
    }
}

class Table(val id : String, val cols : List[String], rows : ListBuffer[TableRow]) {
    def add(row : TableRow) : Unit = {
        this.rows += row
    }
    
    def getRows() : List[TableRow] = {
        return this.rows.toList
    }
    
    def prettyPrint() : Unit = {        
        def format(table: Seq[Seq[Any]]) = table match {
            case Seq() => ""
            case _ => 
                val sizes = for (row <- table) yield (for (cell <- row) yield if (cell == null) 0 else cell.toString.length)
                val colSizes = for (col <- sizes.transpose) yield col.max
                val rows = for (row <- table) yield formatRow(row, colSizes)
                formatRows(rowSeparator(colSizes), rows)
        }

        def formatRows(rowSeparator: String, rows: Seq[String]): String = (
            rowSeparator :: 
            rows.head :: 
            rowSeparator :: 
            rows.tail.toList ::: 
            rowSeparator :: 
            List()).mkString("\n")

        def formatRow(row: Seq[Any], colSizes: Seq[Int]) = {
            val cells = (for ((item, size) <- row.zip(colSizes)) yield if (size == 0) "" else ("%" + size + "s").format(item))
            cells.mkString("|", "|", "|")
        }

        def rowSeparator(colSizes: Seq[Int]) = colSizes map { "-" * _ } mkString("+", "+", "+")

        val lines : ListBuffer[Seq[Any]] = new ListBuffer()
        lines += this.cols
        this.rows.foreach(row => {
            val cells : ListBuffer[Any] = new ListBuffer()
            for (col <- this.cols) {
                cells += row.get(col)
            }
            lines += cells
        })        
        println(format(lines))
    }
}
object Table {
    def create(id : String, cols : List[String], rows : List[TableRow] = List()) : Table = {
        return new Table(id, cols, rows.to[ListBuffer])
    }
}

import scala.collection.mutable.ListBuffer


defined class TableRow
defined object TableRow
defined class Table
defined object Table

To create a table row, we simply call `TableRow.create` on a map from column names to values. Below we (mostly) re-create the rows of our **customer** table from above.

In [7]:
val anders = TableRow.create(Map(
    "id" -> "1", 
    "name" -> "Maria Anders", 
    "city" -> "Berlin", 
    "country" -> "Germany"))
val trujillo = TableRow.create(Map(
    "id" -> "2", 
    "name" -> "Ana Trujillo", 
    "city" -> "Mexico City", 
    "country" -> "Mexico"))
val moreno = TableRow.create(Map(
    "id" -> "3", 
    "name" -> "Antonio Moreno", 
    "city" -> "Mexico City", 
    "country" -> "Mexico"))
val hardy = TableRow.create(Map(
    "id" -> "4", 
    "name" -> "Thomas Hardy", 
    "city" -> "London", 
    "country" -> "UK"))
val berglund = TableRow.create(Map(
    "id" -> "5", 
    "name" -> "Christina Berglund", 
    "city" -> "Luleå", 
    "country" -> "Sweden"))

anders: TableRow = $sess.cmd5Wrapper$Helper$TableRow@74b53e0d
trujillo: TableRow = $sess.cmd5Wrapper$Helper$TableRow@24ecb864
moreno: TableRow = $sess.cmd5Wrapper$Helper$TableRow@6f92a17b
hardy: TableRow = $sess.cmd5Wrapper$Helper$TableRow@1f3b5114
berglund: TableRow = $sess.cmd5Wrapper$Helper$TableRow@42a341a8

We can create new tables by calling `Table.create` and passing in the table name, list of column names, and list of `TableRow` objects. We can also `prettyPrint` to examine the contents of the table.

In [8]:
val customerTable = Table.create(
    "customer", 
    List("id", "name", "city", "country"),
    List(anders, trujillo, moreno, hardy, berglund))

customerTable.prettyPrint()

+--+------------------+-----------+-------+
|id|              name|       city|country|
+--+------------------+-----------+-------+
| 1|      Maria Anders|     Berlin|Germany|
| 2|      Ana Trujillo|Mexico City| Mexico|
| 3|    Antonio Moreno|Mexico City| Mexico|
| 4|      Thomas Hardy|     London|     UK|
| 5|Christina Berglund|      Luleå| Sweden|
+--+------------------+-----------+-------+


customerTable: Table = $sess.cmd5Wrapper$Helper$Table@465d6c45

### Exercise: extend SQL engine

Below is the initial implementation to our SQL engine. `SQLEngine.evalTableExpr` takes a `TableExpr` and a `context` that stores our existing tables. By default, we assume that `context` consists of just our `customerTable` defined above.

So far, `SQLEngine.evalTableExpr` can evaluate simple `SELECT` queries, e.g., `SELECT id, name, city FROM customer`. Your task is to extend `SQLEngine` so that it can execute queries with `WHERE` clauses, e.g., `SELECT id, name, city FROM customer WHERE country = "Mexico"`.

To do so, we need to be able to evaluate the conditional expressions specified within a `WHERE` clause. We have started the initial implementation in `SQLEngine.evalCondExpr`. You will need to extend `SQLEngine.evalCondExpr` to support the `Eq` case.

> Note the return type of `SQLEngine.evalCondExpr`. When we say `WHERE country = "Mexico"`, we are not specifying a single true-or-false value, but rather a function that we apply to each table row to determine if it is acceptable. Hence, the output of evaluating a conditional expression is a function of type `TableRow => Boolean`.

In [16]:
val existingTables : Map[String, Table] = Map(customerTable.id -> customerTable)

object SQLEngine {
    
    def evalTableExpr(t : TableExpr, context : Map[String, Table] = existingTables) : Table = {
        t match {
            case TableIdentifier(s) => context(s)
            // no where clause
            case SelectQuery(columns, fromTable, None) => {
                val table = evalTableExpr(fromTable)
                // row.selectColumn returns a new row containing only the passed columns
                val newRows = table.getRows().map(row => row.selectColumns(columns))
                return Table.create("", columns, newRows)
            }
            // some where clause
            case SelectQuery(columns, fromTable, Some(condExpr)) => {
                // YOUR CODE HERE
                val table = evalTableExpr(fromTable)
                val newRows = table.getRows().filter(evalCondExpr(condExpr)).map(row => row.selectColumns(columns))
                return Table.create("", columns, newRows)
            }
        }
    }

    def evalCondExpr(c : CondExpr) : TableRow => Boolean = c match {
        case ConstTrue => { return (row => true) }
        case ConstFalse => { return (row => false) }
        case Eq(columnName, value) => {
            // YOUR CODE HERE
            return (row => row.get(columnName) == value)
        }
    }
}

existingTables: Map[String, Table] = Map("customer" -> $sess.cmd5Wrapper$Helper$Table@465d6c45)
defined object SQLEngine

In [17]:
// SELECT id, name, city FROM customer
val query1 = new SelectQuery(List("id", "name", "city"), new TableIdentifier("customer"), None)
val tableResult1 = SQLEngine.evalTableExpr(query1)
println("SELECT id, name, city FROM customer")
tableResult1.prettyPrint()

SELECT id, name, city FROM customer
+--+------------------+-----------+
|id|              name|       city|
+--+------------------+-----------+
| 1|      Maria Anders|     Berlin|
| 2|      Ana Trujillo|Mexico City|
| 3|    Antonio Moreno|Mexico City|
| 4|      Thomas Hardy|     London|
| 5|Christina Berglund|      Luleå|
+--+------------------+-----------+


query1: SelectQuery = SelectQuery(List(id, name, city),TableIdentifier(customer),None)
tableResult1: Table = $sess.cmd5Wrapper$Helper$Table@3da7ce23

In [18]:
// SELECT id, name, city FROM customer WHERE true
val query2 = new SelectQuery(List("id", "name", "city"), new TableIdentifier("customer"), Some(ConstTrue))
val tableResult2 = SQLEngine.evalTableExpr(query2)
println("SELECT id, name, city FROM customer WHERE true")
tableResult2.prettyPrint()

SELECT id, name, city FROM customer WHERE true
+--+------------------+-----------+
|id|              name|       city|
+--+------------------+-----------+
| 1|      Maria Anders|     Berlin|
| 2|      Ana Trujillo|Mexico City|
| 3|    Antonio Moreno|Mexico City|
| 4|      Thomas Hardy|     London|
| 5|Christina Berglund|      Luleå|
+--+------------------+-----------+


query2: SelectQuery = SelectQuery(List(id, name, city),TableIdentifier(customer),Some(ConstTrue))
tableResult2: Table = $sess.cmd5Wrapper$Helper$Table@4fcb0ea0

In [19]:
// SELECT id, name, city FROM customer WHERE country = "Mexico"
val query3 = new SelectQuery(List("id", "name", "city"), new TableIdentifier("customer"), Some(new Eq("country", "Mexico")))
val tableResult3 = SQLEngine.evalTableExpr(query3)
println("SELECT id, name, city FROM customer WHERE country = \"Mexico\"")
tableResult3.prettyPrint()

SELECT id, name, city FROM customer WHERE country = "Mexico"
+--+--------------+-----------+
|id|          name|       city|
+--+--------------+-----------+
| 2|  Ana Trujillo|Mexico City|
| 3|Antonio Moreno|Mexico City|
+--+--------------+-----------+


query3: SelectQuery = SelectQuery(List(id, name, city),TableIdentifier(customer),Some(Eq(country,Mexico)))
tableResult3: Table = $sess.cmd5Wrapper$Helper$Table@76e62081

## BSTs
Recal the definition of binary search trees from lecture:

In [20]:
sealed trait NumTree
case object Leaf extends NumTree
case class Node(n: Int, left: NumTree, right: NumTree) extends NumTree

defined trait NumTree
defined object Leaf
defined class Node

### Exercise: insert

For this problem, we want you to implement an `insert` function, which will take a number and a search tree and return a new tree with the number inserted. There should be __no duplicates__, meaning if a number is already in the tree it should not be inserted again.

We will use the following definition of BSTs for this question: $$\max(left) < d < \min(right)$$

In [31]:
// YOUR CODE HERE
def insert(n: Int, t: NumTree): NumTree = t match {
    case Leaf => Node(n, Leaf, Leaf)
    case Node(x, left, right) => {
        if (n < x){
            Node(x, insert(n, left), right)
        } else if (n > x) {
            Node(x, left, insert(n, right))
        } else {
            t
        }
    }
}

defined function insert

In [32]:
val empty = Leaf
assert(insert(5, empty) == Node(5, Leaf, Leaf), "Insert into empty tree failed")

val left_only = Node(4,
                     Node(3,
                          Leaf,
                          Leaf),
                     Leaf)
val inserted = Node(4,
                    Node(3,
                         Leaf,
                         Leaf),
                    Node(5,
                         Leaf,
                         Leaf))
val inserted6 = Node(4,
                     Node(3,
                          Leaf,
                          Leaf),
                     Node(5,
                          Leaf,
                          Node(6,
                               Leaf,
                               Leaf)))

assert(insert(5, left_only) == inserted, "Insert into `left_only` tree failed")

assert(insert(6, inserted) == inserted6, "Insert into `inserted` tree failed")

assert(insert(6, inserted6) == inserted6, "Insertion of existing element failed")

empty: Leaf.type = Leaf
left_only: Node = Node(4,Node(3,Leaf,Leaf),Leaf)
inserted: Node = Node(4,Node(3,Leaf,Leaf),Node(5,Leaf,Leaf))
inserted6: Node = Node(4,Node(3,Leaf,Leaf),Node(5,Leaf,Node(6,Leaf,Leaf)))